<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://profession.ai/datasets/dessert.zip
!unzip -qq dessert.zip

--2020-06-26 12:31:47--  https://profession.ai/datasets/dessert.zip
Resolving profession.ai (profession.ai)... 13.35.153.5, 13.35.153.89, 13.35.153.51, ...
Connecting to profession.ai (profession.ai)|13.35.153.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232148910 (221M) [application/zip]
Saving to: ‘dessert.zip’

dessert.zip         100%[===================>] 221.39M  24.3MB/s    in 11s     

2020-06-26 12:31:58 (20.7 MB/s) - ‘dessert.zip’ saved [232148910/232148910]



In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [19]:
DATASET_DIR = "dessert"
BATCH_SIZE = 64

TOT_IMAGES = 10000 
IMG_SIZE = (228, 228)

In [20]:
datagen = ImageDataGenerator(validation_split=0.1, rescale=1./255,)

train_generator = datagen.flow_from_directory(
        DATASET_DIR, 
        target_size=IMG_SIZE, 
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training')

test_generator = datagen.flow_from_directory(
        DATASET_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation')

print(train_generator.class_indices)
print(len(train_generator))

Found 9000 images belonging to 5 classes.
Found 1000 images belonging to 5 classes.
{'cannoli': 0, 'gelato': 1, 'panna_cotta': 2, 'tiramisu': 3, 'torta_di_mele': 4}
141


In [25]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Dropout(0.4))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(5, activation='softmax'))

print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 228, 228, 32)      896       
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 228, 228, 32)      9248      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 76, 76, 32)        0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 76, 76, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 76, 76, 32)        9248      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 76, 76, 32)        9248      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 25, 25, 32)       

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.fit(
        train_generator,
        steps_per_epoch=int(TOT_IMAGES*0.9 // BATCH_SIZE),
        epochs=100)

Epoch 1/100
140/140 [==============================] - 41s 293ms/step - loss: 1.6186 - accuracy: 0.1979
Epoch 2/100
140/140 [==============================] - 41s 293ms/step - loss: 1.6097 - accuracy: 0.1955
Epoch 3/100
140/140 [==============================] - 41s 293ms/step - loss: 1.6096 - accuracy: 0.2099
Epoch 4/100
140/140 [==============================] - 41s 290ms/step - loss: 1.6098 - accuracy: 0.1955
Epoch 5/100
140/140 [==============================] - 41s 291ms/step - loss: 1.6098 - accuracy: 0.1939
Epoch 6/100
140/140 [==============================] - 40s 289ms/step - loss: 1.6099 - accuracy: 0.1987
Epoch 7/100
140/140 [==============================] - 40s 289ms/step - loss: 1.6095 - accuracy: 0.2006
Epoch 8/100
140/140 [==============================] - 41s 290ms/step - loss: 1.6099 - accuracy: 0.1967
Epoch 9/100
140/140 [==============================] - 40s 288ms/step - loss: 1.6097 - accuracy: 0.1944
Epoch 10/100
140/140 [==============================] - 40s 289m

In [24]:
metrics_train = model.evaluate(train_generator)
metrics_test = model.evaluate(test_generator)

print("Train Accuracy = %.4f - Train Loss = %.4f" % (metrics_train[1], metrics_train[0]))
print("Test Accuracy = %.4f - Test Loss = %.4f" % (metrics_test[1], metrics_test[0]))

16/16 [==============================] - 3s 211ms/step - loss: 1.2819 - accuracy: 0.4770
Train Accuracy = 0.6358 - Train Loss = 1.0308
Test Accuracy = 0.4770 - Test Loss = 1.2819
